# Mall_OS – Constitutional Log Bible

This notebook treats `constitution_log.jsonl` as the **living game bible**.

- Every law signed by the Governor is logged as a JSONL entry.
- Each entry contains:
  - law metadata (id, title, scope, effects)
  - QBIT-based political power metrics
  - law strength & interpretation radius
  - a snapshot of the world QBIT state at the moment of passage

Use this notebook to:
- query laws by zone, time, and QBIT conditions
- visualize how the mall's political landscape evolves
- drive auto-construction and behavior spec generation

In [ ]:
# Config & Imports
import json
from pathlib import Path
from typing import List, Dict, Any

import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# Path to your constitution log
CONSTITUTION_LOG = Path("../data/constitution_log.jsonl")

pd.set_option("display.max_colwidth", 120)

In [ ]:
# Load the JSONL Log
def load_constitution_log(path: Path = CONSTITUTION_LOG) -> pd.DataFrame:
    rows = []
    if not path.exists():
        raise FileNotFoundError(f"Constitution log not found at {path}")

    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            law = obj.get("law", {})
            effects = law.get("effects", {})
            scope = law.get("scope", [])
            qbit_snapshot = obj.get("qbit_snapshot", {})

            rows.append({
                "law_id": law.get("law_id"),
                "title": law.get("title"),
                "scope": scope,
                "effects": effects,
                "political_power": obj.get("political_power"),
                "pass_probability": obj.get("pass_probability"),
                "strength": obj.get("strength"),
                "interpretation_radius": obj.get("interpretation_radius"),
                "qbit_snapshot": qbit_snapshot,
                "timestamp_real": obj.get("timestamp_real"),
            })
    return pd.DataFrame(rows)

df = load_constitution_log()
print(f"Loaded {len(df)} constitutional entries")
df.head()

In [ ]:
# Helper Filters
def filter_by_zone(df: pd.DataFrame, zone_id: str) -> pd.DataFrame:
    """Get all laws affecting a specific zone."""
    return df[df["scope"].apply(lambda scopes: any(zone_id in s for s in scopes))]

def filter_by_law_id(df: pd.DataFrame, law_id: str) -> pd.DataFrame:
    """Get specific law by ID."""
    return df[df["law_id"] == law_id]

def filter_strong_laws(df: pd.DataFrame, min_strength: float = 1.0) -> pd.DataFrame:
    """Get laws with high strength (strict enforcement)."""
    return df[df["strength"] >= min_strength]

def filter_high_wiggle_laws(df: pd.DataFrame, min_radius: float = 0.5) -> pd.DataFrame:
    """Get laws with high interpretation freedom."""
    return df[df["interpretation_radius"] >= min_radius]

# Example use:
food_court_laws = filter_by_zone(df, "ZONE:FOOD_COURT")
print(f"\nLaws affecting Food Court: {len(food_court_laws)}")
food_court_laws[["law_id", "title", "strength", "interpretation_radius"]]

In [ ]:
# Timeline View
df_sorted = df.sort_values("timestamp_real")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Law strength over time
ax1.plot(df_sorted["timestamp_real"], df_sorted["strength"], marker="o", linewidth=2)
ax1.set_xlabel("Time")
ax1.set_ylabel("Law Strength")
ax1.set_title("Law Strength Over Time")
ax1.grid(True, alpha=0.3)

# Interpretation radius over time
ax2.plot(df_sorted["timestamp_real"], df_sorted["interpretation_radius"], 
         marker="s", color="green", linewidth=2)
ax2.set_xlabel("Time")
ax2.set_ylabel("Interpretation Radius (wiggle room)")
ax2.set_title("Interpretation Freedom Over Time")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## "What Was the World Like When This Law Was Born?"

This is the bible function: query the exact QBIT state when a law passed.

In [ ]:
def inspect_law_snapshot(df: pd.DataFrame, law_id: str):
    """Inspect the complete context when a law was enacted."""
    row = filter_by_law_id(df, law_id)
    if row.empty:
        print(f"No law with id {law_id}")
        return
    row = row.iloc[0]
    
    print("="*70)
    print(f"LAW: {row['law_id']} :: {row['title']}")
    print("="*70)
    print()
    print("Scope:", row["scope"])
    print()
    print("Political Metrics:")
    print(f"  Political Power: {row['political_power']:.3f}")
    print(f"  Pass Probability: {row['pass_probability']:.3f}")
    print(f"  Strength: {row['strength']:.3f}")
    print(f"  Interpretation Radius: {row['interpretation_radius']:.3f}")
    print()
    print("Effects:")
    pprint(row["effects"])
    print()
    print("QBIT Snapshot at Passage:")
    pprint(row["qbit_snapshot"])
    print()
    print("="*70)
    print("INTERPRETATION GUIDANCE FOR AI CONSTRUCTORS:")
    print("="*70)
    if row['interpretation_radius'] < 0.3:
        print("⚖️  STRICT ENFORCEMENT")
        print("   Low wiggle room. Follow effects literally.")
        print("   Creative interpretation not recommended.")
    elif row['interpretation_radius'] < 0.6:
        print("⚖️  MODERATE FLEXIBILITY")
        print("   Some wiggle room. Edge cases can be interpreted.")
        print("   Stay aligned with QBIT weights.")
    else:
        print("⚖️  HIGH CREATIVE FREEDOM")
        print("   Significant wiggle room. Creative interpretation allowed.")
        print("   Law is contested/weak - use judgment.")
    print()

# Example usage (replace with actual law_id from your log)
if len(df) > 0:
    inspect_law_snapshot(df, df.iloc[0]['law_id'])

## Export for Auto-Construction

Build zone contracts that downstream tools can consume.

In [ ]:
def build_zone_contract(df: pd.DataFrame, zone_id: str) -> Dict[str, Any]:
    """
    Build a construction contract for a zone.
    
    This is what auto-constructors / AI behavior trees consume:
    - Which laws apply to this zone
    - How strictly to enforce each
    - What creative freedom exists
    """
    zone_laws = filter_by_zone(df, zone_id).sort_values("timestamp_real")
    
    contract = {
        "zone_id": zone_id,
        "laws": [],
        "meta": {
            "total_laws": len(zone_laws),
            "average_strength": zone_laws["strength"].mean() if len(zone_laws) > 0 else 0,
            "average_wiggle": zone_laws["interpretation_radius"].mean() if len(zone_laws) > 0 else 0,
        }
    }

    for _, row in zone_laws.iterrows():
        contract["laws"].append({
            "law_id": row["law_id"],
            "title": row["title"],
            "strength": row["strength"],
            "interpretation_radius": row["interpretation_radius"],
            "effects": row["effects"],
        })
    
    return contract

# Example: Build contract for Food Court
zone_contract = build_zone_contract(df, "ZONE:FOOD_COURT")
print("Food Court Construction Contract:")
print(json.dumps(zone_contract, indent=2))

In [ ]:
# Export all zone contracts
def export_all_zone_contracts(df: pd.DataFrame, output_dir: Path = Path("../data/zone_contracts")):
    """
    Generate construction contracts for all zones.
    
    Auto-constructors load these to know:
    - spawn gates, signage, cameras
    - adjust NPC spawn tables
    - tweak lighting, clutter, access routes
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Extract all unique zones from scopes
    all_zones = set()
    for scopes in df["scope"]:
        for scope in scopes:
            if "ZONE:" in scope:
                all_zones.add(scope)
    
    contracts = {}
    for zone in sorted(all_zones):
        contract = build_zone_contract(df, zone)
        contracts[zone] = contract
        
        # Save individual contract
        zone_name = zone.replace("ZONE:", "").lower()
        output_path = output_dir / f"{zone_name}_contract.json"
        with open(output_path, "w") as f:
            json.dump(contract, f, indent=2)
        print(f"✓ Exported {zone} → {output_path}")
    
    # Save master index
    index_path = output_dir / "_index.json"
    with open(index_path, "w") as f:
        json.dump({
            "zones": list(all_zones),
            "total_contracts": len(contracts),
            "generated_at": pd.Timestamp.now().isoformat()
        }, f, indent=2)
    print(f"\n✓ Master index → {index_path}")
    
    return contracts

# Export contracts for auto-construction pipeline
if len(df) > 0:
    contracts = export_all_zone_contracts(df)

## Notebook as Bible Index

### Using this Notebook as the Game Bible

This notebook *is* the Mall_OS Constitution:

- Every row in `df` is a canonized law event.
- Helper functions let you query:
  - by zone (Food Court, Arcade, Service Corridors…)
  - by law strength (iron rules vs. soft guidelines)
  - by interpretation radius (how creatively NPCs can bend it)
- The zone contracts produced here feed the **auto-construction pipeline**:
  - voxel painting rules
  - constructor behavior constraints
  - Synthactor spawn permissions
  - signage & environmental storytelling

In future versions:
- Add rich visualizations for QBIT drift over time.
- Embed Sora / video stills for key legislative moments.
- Attach NPC transcripts when laws were debated in-world.

## Governor's Interpretation Workspace

**This section is MUTABLE** - Governor adds interpretations within wiggle room.

The cells above are IMMUTABLE (archive of constitutional log).

Below this line: Governor's creative interpretations within `interpretation_radius` constraints.

---

### GOVERNOR RULING: LC_0231 Edge Case Example

**Law:** Food Court Curfew  
**Interpretation Radius:** 0.60 (moderate flexibility)

**Case:** What if a customer is still eating at 10:01pm?

**Ruling:**  
The law states "after hours only maintenance and security."

Given interpretation_radius = 0.60, I exercise creative interpretation:

- "Security" can include persons under security escort (flexible reading)
- Customer finishing meal at 10:01pm is **ALLOWED** if:
  1. Security is present
  2. Customer is actively eating (not loitering)
  3. Customer exits within 15 minutes

**Wiggle Used:** 0.4 of 0.60 available

**Alignment Check:**
- ✓ QBIT weight: heat=-0.4 (reduces activity) - Still reduces overall activity
- ✓ Political power: 0.67 (strong support) - Doesn't undermine law's support
- ✓ Core effect: npc_density_max=0.35 - Complies (one customer << 0.35)

**AI Constructor Guidance:**  
When generating NPC behavior for late customers:
```python
if time > curfew_time and npc.is_eating:
    if security_present and meal_almost_done:
        allow_with_timer(15_minutes)
    else:
        enforce_exit()
```

**Logged:** This ruling becomes precedent for future edge cases.

---

In [ ]:
# Log governor's interpretation to structured data
interpretations = {
    "LC_0231": [
        {
            "date": "2025-12-07",
            "case": "Customer finishing meal after curfew",
            "ruling": "Allowed under security escort, must exit within 15 min",
            "wiggle_used": 0.4,
            "wiggle_available": 0.60,
            "rationale": "Aligns with law's heat reduction goal while allowing completion of legitimate activity",
            "ai_guidance": "if time > curfew and is_eating and security_present: allow_with_timer(15min)"
        }
    ]
}

# Save interpretations for AI constructors to query
interpretation_log_path = Path("../data/governor_interpretations.json")
with open(interpretation_log_path, "w") as f:
    json.dump(interpretations, f, indent=2)

print(f"✓ Governor interpretations saved to {interpretation_log_path}")
print("\nAI constructors can now query:")
print("  - Immutable: constitution_log.jsonl (what the law IS)")
print("  - Mutable: governor_interpretations.json (how to interpret edge cases)")

## Summary

**This notebook serves three purposes:**

1. **Archive** (IMMUTABLE)  
   Query constitutional history: "What was the world like when this law passed?"

2. **Analysis** (COMPUTED)  
   Visualize political landscape, law strength evolution, zone contracts

3. **Workspace** (MUTABLE)  
   Governor adds interpretations within `interpretation_radius` constraints

**AI Constructors query both layers:**
- `constitution_log.jsonl` → What is the law? (immutable truth)
- `governor_interpretations.json` → How to handle edge cases? (constrained creativity)

**The filing cabinet is complete.**